In [2]:
from urllib import request

EXTRACTION_FOLDER_PATH = './work/'

def get_path_for_csv_file():
    source_csv_filename = 'votacao_candidato_munzona_2022_BRASIL.csv'
    extracted_file = os.path.join(EXTRACTION_FOLDER_PATH, 'extracted', source_csv_filename)

    return extracted_file


def check_if_extraction_folder_is_created(folder_path: str):
    folder_is_created = os.path.exists(folder_path)

    if not folder_is_created:
      os.mkdir(folder_path)


def download_csv_if_not_exists(csv_path: str):
   work_dir = EXTRACTION_FOLDER_PATH
   destination_file = "votacao_candidato_munzona_2022.zip"
   url = 'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_2022.zip'
   destination_zip_path = os.path.join(work_dir, destination_file)

   zip_already_downloaded = os.path.exists(destination_zip_path)

   if not zip_already_downloaded:
       download_zip_file(url, destination_zip_path)

   file_is_extracted = os.path.exists(csv_path)

   if not file_is_extracted:
        source_csv_filename = 'votacao_candidato_munzona_2022_BRASIL.csv'

        csv_extracted_path = os.path.join(work_dir, 'extracted')

        unzip_file(source_csv_filename, destination_zip_path, csv_extracted_path)


def download_zip_file(url: str, destination_path: str):
    request.urlretrieve(url, destination_path)


def unzip_file(file: str, source_path: str, extract_path):
    from zipfile import ZipFile

    with ZipFile(source_path, 'r') as zip_file:
        zip_file.extract(file, extract_path)

In [3]:
csv_file_path = get_path_for_csv_file()
check_if_extraction_folder_is_created(EXTRACTION_FOLDER_PATH)

print(" ⌛ Baixando arquivo .zip e extraindo .csv...")

download_csv_if_not_exists(csv_file_path)

 ⌛ Baixando arquivo .zip e extraindo .csv...


In [1]:
import duckdb
import pandas as pd

connection = duckdb.connect()

csv_file_path = './work/extracted/votacao_candidato_munzona_2022_BRASIL.csv'

chunk_size = 10000  # Number of rows per chunk
chunks = pd.read_csv(csv_file_path, encoding='latin1', chunksize=chunk_size, on_bad_lines='skip')

# Initialize an empty DuckDB table with the first chunk
for i, chunk in enumerate(chunks):
    if i == 0:
        # Create the table and insert the first chunk
        connection.register('temp_chunk', chunk)
        connection.execute("CREATE TABLE my_table AS SELECT * FROM temp_chunk")
    else:
        # Append subsequent chunks
        connection.register('temp_chunk', chunk)
        connection.execute("INSERT INTO my_table SELECT * FROM temp_chunk")

# connection.execute(f"""


print(connection.execute("DESCRIBE my_table").fetchall())

connection.close()

[('DT_GERACAO;"HH_GERACAO";"ANO_ELEICAO";"CD_TIPO_ELEICAO";"NM_TIPO_ELEICAO";"NR_TURNO";"CD_ELEICAO";"DS_ELEICAO";"DT_ELEICAO";"TP_ABRANGENCIA";"SG_UF";"SG_UE";"NM_UE";"CD_MUNICIPIO";"NM_MUNICIPIO";"NR_ZONA";"CD_CARGO";"DS_CARGO";"SQ_CANDIDATO";"NR_CANDIDATO";"NM_CANDIDATO";"NM_URNA_CANDIDATO";"NM_SOCIAL_CANDIDATO";"CD_SITUACAO_CANDIDATURA";"DS_SITUACAO_CANDIDATURA";"CD_DETALHE_SITUACAO_CAND";"DS_DETALHE_SITUACAO_CAND";"TP_AGREMIACAO";"NR_PARTIDO";"SG_PARTIDO";"NM_PARTIDO";"NR_FEDERACAO";"NM_FEDERACAO";"SG_FEDERACAO";"DS_COMPOSICAO_FEDERACAO";"SQ_COLIGACAO";"NM_COLIGACAO";"DS_COMPOSICAO_COLIGACAO";"ST_VOTO_EM_TRANSITO";"QT_VOTOS_NOMINAIS";"NM_TIPO_DESTINACAO_VOTOS";"QT_VOTOS_NOMINAIS_VALIDOS";"CD_SIT_TOT_TURNO";"DS_SIT_TOT_TURNO"', 'VARCHAR', 'YES', None, None, None)]
